In [7]:
alphavalue(M, 3e-16)

<Quantity 0.22449414>

In [8]:
import os, sys
os.chdir("../")
path = os.getcwd()
sys.path.insert(0, path)
import GWGen
from GWGen.Utils import *
from GWGen.WFGenerator import *
import matplotlib.pyplot as plt
import scipy as sp
import scipy.signal
import superrad
import pyfftw


# set initial parameters
M = 1e5
m = 1e1
mu = 3e-16
e0 = 0.6
p0 = GetInitialP(M,e0)
Phi_phi0 = 0.
Phi_theta0 =0.
Phi_r0 = 0.


a=0.75 #SMBH Spin
Y0=1. #Initial Inclincation
qS=np.pi/4 #Sky Location Polar Angle in solar system barycenter coordinate system
phiS=0. #Sky Location Azimuthal Angle in solar system barycenter coordinate system
qK=np.pi/4 #Initial BH Spin Polar Angle in solar system barycenter coordinate system
phiK=0. #Initial BH Spin Azimuthal Angle in solar system barycenter coordinate system
dist=1. #Distance to source (Mpc)
mich=False #assume LISA long baseline response approximation

T=5.8 #LISA data run is 5 years. We set the max time to be longer because the proca cloud extends the inspiral time
dt=15 #time resolution in seconds

alphaval = alphavalue(M,mu)

use_gpu = False

# keyword arguments for inspiral generator (RunKerrGenericPn5Inspiral)
insp_kwargs = {
    "npoints": 110,  # we want a densely sampled trajectory
    "max_init_len": int(1e4),  # all of the trajectories will be well under len = 1000
    "dense_output":True
}

# keyword arguments for summation generator (AAKSummation)
sum_kwargs = {
    "use_gpu": use_gpu,  # GPU is availabel for this type of summation
    "pad_output": False,
}
def testinnerprod(td, wv1, wv2, maximize=False):
    if maximize:
        subresults = []

        for i in range(5):
            wv2 = wv2 * np.exp(1j* (i/5) * 2 * np.pi)
            pyfftw.config.NUM_THREADS = multiprocessing.cpu_count()
            pyfftw.interfaces.cache.enable()
            sp.fft.set_backend(pyfftw.interfaces.scipy_fft)
            responses_h1 = list(detector_response(wv1, viewingangles = [qS, phiS,0]).values())
            responses_h2 = list(detector_response(wv2, viewingangles = [qS, phiS,0]).values())
            h1resp_f = sp.fft.fft(responses_h1)[:,:]
            h2resp_f = sp.fft.fft(responses_h2)[:,:]
            h2resp_f_star = np.conjugate(h2resp_f)
            freq_range = sp.fft.fftfreq(int(len(td)), d = float(td[1]-td[0]))[:]
            psd = LisaSensitivity(np.abs(freq_range))
            Factor1 = sp.fft.ifft(h1resp_f/psd)
            Factor2 = sp.fft.ifft(h2resp_f_star)
            convolutions = sp.signal.convolve(Factor1, Factor2, method="fft", mode="full")[[0,-1]].real
            combined_convolutions = np.sum(convolutions,axis=0)
            print("middle inx: ", len(combined_convolutions)/2)
            print("middle val: ", combined_convolutions[int(np.floor(len(combined_convolutions)/2))])
            print("arg max: ", np.argmax(combined_convolutions))
            print("max: ", max(combined_convolutions))
            print(' ')
            subresults.append(max(combined_convolutions))
        return max(subresults), subresults
    else:
        h1_responses = list(detector_response(wv1, viewingangles = [qS, phiS,0]).values())
        h2_responses = list(detector_response(wv2, viewingangles = [qS, phiS,0]).values())
        pyfftw.config.NUM_THREADS = multiprocessing.cpu_count()
        pyfftw.interfaces.cache.enable()
        with sp.fft.set_backend(pyfftw.interfaces.scipy_fft):
            with sp.fft.set_workers(os.cpu_count() + 2):
                if not use_gpu:
                    h1resp_f = sp.fft.fft(h1_responses)[:,:] #drop zero frequency
                    h2resp_f = sp.fft.fft(h2_responses)[:,:]
                    h2resp_f_star = np.conjugate(h2resp_f)
                    timelength = len(td)
                    DeltaT = td[1]-td[0]
                    frequency_range = sp.fft.fftfreq(int(timelength), d=float(DeltaT))[:]
                else:
                    h1resp_f = xp.fft.fft(h1_responses)[:,:] #drop zero frequency
                    h2resp_f = xp.fft.fft(h2_resonses)[:,:]
                    h2resp_f_star = np.conjugate(h2resp_f)
                    timelength = len(td)
                    DeltaT = td[1]-td[0]
                    frequency_range = xp.fft.fftfreq(int(timelength), d=float(DeltaT))[:]
                PowerSpectralDensity = LisaSensitivity(np.abs(frequency_range))
                Factor1 = sp.fft.ifft(h1resp_f/PowerSpectralDensity)
                Factor2 = sp.fft.ifft(h2resp_f_star)
                if use_gpu:
                    Factor1 = Factor1.get()
                    Factor2 = Factor2.get()
                convolutions = sp.signal.convolve(Factor1, Factor2, method="fft", mode="full")[[0,-1]].real
                combined_convolutions = np.sum(convolutions,axis=0)
                

                fullconv = max(combined_convolutions)
                return fullconv
            
WithoutProcaInspiralKwargs = insp_kwargs.copy()
WithoutProcaSumKwargs=sum_kwargs.copy()
withoutprocagen = EMRIWaveform(inspiral_kwargs=WithoutProcaInspiralKwargs, sum_kwargs=WithoutProcaSumKwargs, use_gpu=False)
print(r"alpha = {0}".format(alphaval))
print("initial p = {0}".format(p0))

alpha = 0.22449414184716673
initial p = 33.35


In [9]:
ulb = superrad.ultralight_boson.UltralightBoson(spin=1,model="relativistic")
withprocagen = EMRIWithProcaWaveform(inspiral_kwargs=insp_kwargs.copy(),sum_kwargs=sum_kwargs.copy())
wv1 = withoutprocagen(M, m, a, p0, e0, Y0, qS, phiS, qK, phiK, dist,Phi_phi0=Phi_phi0, Phi_theta0=Phi_theta0, Phi_r0=Phi_r0, mich=mich, dt=dt, T=T)
#wv2 = withoutprocagen(M, m, a, 10, e0, Y0, qS, phiS, qK, phiK, dist,Phi_phi0=Phi_phi0, Phi_theta0=Phi_theta0, Phi_r0=Phi_r0, mich=mich, dt=dt, T=T)
wv2 = withprocagen(M,m,mu,a,p0,e0,Y0,T=T,qS=qS,phiS=phiS,qK=qK,phiK=phiK,dist=dist,mich=mich, UltralightBoson=ulb)

"""
if len(wv1)<len(wv2):
    wv1 = np.pad(wv1, (0,len(wv2)-len(wv1)))
elif len(wv2)<len(wv1):
    wv2= np.pad(wv2, (0,len(wv1)-len(wv2)))
"""
minlen = min([len(wv1), len(wv2)])
td = np.arange(minlen)*dt
wv1 = wv1[:minlen]
wv2 = wv2[:minlen]

In [10]:
faithdir = Faithfulness(td,wv1,wv2, viewingangle = [qS,phiS,0])
faithdir

0.027837779046805778

In [ ]:
leng = int(1000000)
testwv = wv1[:]
tdtest = td[:]
t12 = testinnerprod(tdtest, testwv, testwv, maximize=True)
t11 = testinnerprod(tdtest, testwv, testwv)
print(t12)
print(t11)

In [ ]:
t12[0]/t11

In [ ]:
leng = int(10000000)
testwv = wv1[:leng]
lisares = detector_response(testwv, viewingangles = [qS,phiS,0])
res1 = lisares["h1"]
wv1f = np.fft.fft(res1)[:]
freqs = np.fft.fftfreq(len(res1))[:]
psd = LisaSensitivity(np.abs(freqs))
H1 = np.fft.ifft(wv1f/psd)
H2 = np.fft.ifft(np.conjugate(wv1f))
conv = sp.signal.convolve(H1,H2, method="fft")
convlen = len(conv)
convlen2 = int(len(conv)/2)
print(convlen)
print(convlen2)
print(conv[convlen2])
print(max(conv))
print(np.argmax(conv))
plt.plot(conv)

In [ ]:
leng = int(1000)
testwv = wv1[:leng]
t12 = testinnerprod(range(leng), testwv, testwv, maximize=True)
t11 = testinnerprod(range(leng), testwv, testwv)
print(t12)
print(t11)
print(t12[0]/t11)

In [ ]:
faithdir

In [ ]:
wvtest = wv2[-100:]
ang = np.angle(wvtest[-1])
wvtestang = wvtest*np.exp(-1j*ang)
plt.plot(wvtest.imag)
plt.plot(wvtestang.real)
plt.plot(wvtestang.imag)

In [ ]:
p = np.real(datf)
newp = np.real(np.exp(-np.sign(freqs)*1j*ang)*datf)
newp[0]=-0.1e-19
plt.plot(freqs,newp,label="datf")
plt.plot(freqs,p,label='dat')
plt.xlim(-0.003,0.003)

In [ ]:
testf = lambda f: np.exp(-f**2 + f)*np.cos(20*f)
testf = lambda f,a: np.sin(50*(f+f**2+ np.cos(f)) + a)/(f+1) 
dat = wv1.imag[-4000:]
dom = np.arange(0,7,0.001)
#dat = testf(dom,0)
freqs = sp.fft.fftfreq(len(dat))
datf = sp.fft.fft(dat)
ang = np.pi/2
shifted_datf = np.exp(-np.sign(freqs)*1j*ang)*datf
shifted_datf[0]=-0.5e-19
ndat = sp.fft.ifft(shifted_datf)
plt.plot(dat[0:-1], label="old")
plt.plot(ndat[0:-1],label="new")
plt.xlim(0,4000)
plt.legend()

In [ ]:
w1=WaveformInnerProduct(td,wv1,wv1)
w2=WaveformInnerProduct(td,wv2,wv2)
w12 = WaveformInnerProduct(td,wv1,wv2, Tmaximize=True)
w12/np.sqrt(w1*w2)

In [ ]:
if len(wv1)<len(wv2):
    wv1 = np.pad(wv1,(0,len(wv2)-len(wv1)))
elif len(wv2)<len(wv1):
    wv2 = np.pad(wv2,(0,len(wv1)-len(wv2)))
wv1f = sp.fft.fft(wv1)[1:]
wv2f = sp.fft.fft(wv2)[1:]
wv2fs = np.conjugate(wv2f)
freqs = sp.fft.fftfreq(int(len(td)), d = td[1]-td[0])[1:]
psd = LisaSensitivity(np.abs(freqs))
H1 = sp.fft.ifft(wv1f/psd)
H2 = sp.fft.ifft(wv2fs)

In [ ]:
res1=[]
import time

for theta in np.linspace(0,2*np.pi,4):
    conv = sp.signal.convolve(H1,H2*np.exp(1j*theta),method="fft", mode="full")
    print(max(np.real(conv)))
    res1.append(max(np.real(conv)))

pconv = sp.signal.convolve(H1,H2,method="fft", mode="full")

In [ ]:
res  =[]
for th in np.linspace(0,2*np.pi,4):
    realval = np.real(pconv*np.exp(1j*th))
    maxinx = np.argmax(realval)
    maxval = realval[maxinx]
    print(maxinx)
    res.append(maxval)

In [ ]:
res

In [ ]:
fai = []
for i in np.linspace(1.e-18,4.5e-18,10):
    print("wv1")
    wv1 = withoutprocagen(M, m, a, p0, e0, Y0, qS, phiS, qK, phiK, dist,Phi_phi0=Phi_phi0, Phi_theta0=Phi_theta0, Phi_r0=Phi_r0, mich=mich, dt=dt, T=T)
    print('wv2')
    wv2 = withprocagen(M,m,i,a,p0,e0,Y0,T=T,qS=qS,phiS=phiS,qK=qK,phiK=phiK,dist=dist,mich=mich, UltralightBoson=ulb)
    
    if len(wv1)<len(wv2):
        wv1 = np.pad(wv1, (0,len(wv2)-len(wv1)))
    elif len(wv2)<len(wv1):
        wv2= np.pad(wv2, (0,len(wv1)-len(wv2)))
    
    minlen = min([len(wv1), len(wv2)])
    td = np.arange(minlen)*dt
    #wv1 = wv1[:minlen]
    #wv2 = wv2[:minlen]
    print("innerproducts")
    res1 = innerprod4(td,wv1,wv2)
    res2 = innerprod4(td,wv1,wv1)
    res3 = innerprod4(td,wv2,wv2)
    fai.append(
        [i,res1,res2,res3]
    )
    print(i)
fai = np.asarray(fai)

In [ ]:
fai = np.asarray(fai)
args = [np.argmax(i) for i in fai[:,1]]
vals = np.array([np.max(i) for i in fai[:,1]])
norms = np.sqrt(np.array(fai[:,3]*fai[:,2],dtype=np.float64))
faiths = vals/norms
plt.scatter(fai[:,0], args)
plt.title("max positions")
plt.show()
plt.scatter(fai[:,0], vals)
plt.title("inner product")
plt.show()
plt.scatter(fai[:,0], faiths)

plt.title("faithfulness")

In [ ]:
def innerprod2(td,w1,w2):
    wv1fft = sp.fft.fft(w1)
    wv2fft = sp.fft.fft(w2)
    freqs = sp.fft.fftfreq(len(td), d=float(td[1]-td[0]))
    leng = np.argmax(freqs)
    wv1fft = wv1fft[1:leng]
    wv2fft = wv2fft[1:leng]
    freqs = freqs[1:leng]
    lisasens = LisaSensitivity(freqs)
    integrand = wv1fft * np.conjugate(wv2fft)/lisasens
    res = 4*sp.integrate.simpson(integrand.real,x=freqs)
    return res

def innerprod3(td,w1,w2):
    wv1fft = sp.fft.fft(w1)
    wv2fft = sp.fft.fft(w2)
    freqs = sp.fft.fftfreq(len(td), d=float(td[1]-td[0]))
    wv1fft = wv1fft[1:]
    wv2fft = wv2fft[1:]
    freqs = freqs[1:]
    lisasens = LisaSensitivity(np.abs(freqs))
    integrand = wv1fft * np.conjugate(wv2fft)/lisasens
    res = 4*sp.integrate.simpson(integrand.real,x=freqs)
    return res

def innerprod1(td, w1,w2):
    wv1fft = sp.fft.fft(w1)[1:]
    wv2fft = sp.fft.fft(w2)[1:]
    freqs = sp.fft.fftfreq(len(td), d=float(td[1]-td[0]))
    lisasens = LisaSensitivity(np.abs(freqs[1:]))
    integrand = wv1fft * np.conjugate(wv2fft)/lisasens
    
    res = 2*np.real(np.fft.ifft(integrand))
    return res

def innerprod4(td,w1,w2):
    wv1fft = sp.fft.fft(w1)
    wv2fft = sp.fft.fft(w2)
    freqs = sp.fft.fftfreq(len(td), d=float(td[1]-td[0]))
    wv1fft = wv1fft[1:]
    wv2fft = wv2fft[1:]
    freqs = freqs[1:]
    lisasens = LisaSensitivity(np.abs(freqs))
    t1 = sp.fft.ifft(wv1fft/lisasens)
    t2 = sp.fft.ifft(np.conjugate(wv2fft))
    conv = sp.signal.convolve(t1,t2,method="fft", mode="full")
    convlen = int(len(conv))+1
    #plt.plot(conv)
    #plt.scatter([convlen/2], [conv[int(convlen/2)]])
    if np.all(w1==w2):
        return 2*np.real(conv[int(convlen/2)])
    return 2*np.real(conv)

In [ ]:
t11 = innerprod1(td,wv1,wv2)
t12 = innerprod1(td,wv1,wv1)
t13 = innerprod1(td,wv2,wv2)
t21 = innerprod2(td,wv1,wv2)
t22 = innerprod2(td,wv1,wv1)
t23 = innerprod2(td,wv2,wv2)
t31 = innerprod3(td,wv1,wv2)
t32 = innerprod3(td,wv1,wv1)
t33 = innerprod3(td,wv2,wv2)

In [ ]:
print(t21)


In [ ]:
Faithfulness(td,wv1,wv2,maximize=False)

In [ ]:
Faithfulness(td,wv1,wv2,maximize=True)

In [ ]:
get_mismatch(wv1,wv2)

In [ ]:
def fun(x):
    sig = 2
    if x**2<sig:
        return np.exp(-1/(sig-x**2))/np.exp(-1/sig)
    else:
        return 0
def fun2(x):
    if np.abs(x)<1:
        return 1
    else:
        return 0
dom = np.arange(-2,2,0.01)
ran = np.array([fun(i) for i in dom])
ran1 = np.array([fun2(i) for i in dom])
conv = sp.signal.convolve(ran,ran1,method="fft")
plt.plot(ran1)
plt.plot(ran)
plt.show()
plt.plot(conv)

In [ ]:
len(ran)

In [ ]:
sp.integrate.simpson(ran*ran1)

In [ ]:
Norm = "backward"
wv1f = np.fft.fft(wv1, norm=Norm)
wv2f = np.fft.fft(wv2, norm=Norm)
fd = np.fft.fftfreq(len(td),d=float(td[1]-td[0]))
fdlen = int(len(fd)/2)
fd = fd[1:fdlen]
wv1ft = wv1f[1:fdlen]
wv2ft = wv2f[1:fdlen]
wv2fstar = np.conjugate(wv2ft)
wv1fstar = np.conjugate(wv1ft)
psd = LisaSensitivity(fd)
integrand = wv1ft*wv2fstar/psd*tophatran
integrandwv1 = wv1ft*wv1fstar/psd*tophatran
integrandwv2 = wv2ft*wv2fstar/psd*tophatran
tophatran = [tophat(x,0.0001,1) for x in fd]

intfft = 4*np.real(np.fft.ifft(wv1ft/psd*wv2fstar*tophatran, norm=Norm))
intfftwv1 =  4*np.real(np.fft.ifft(wv1ft/psd*wv1fstar*tophatran, norm=Norm))
intfftwv2 =  4*np.real(np.fft.ifft(wv2ft/psd*wv2fstar*tophatran, norm=Norm))
integral = 4*sp.integrate.simpson(integrand.real,x=fd)
integralwv1 = 4*sp.integrate.simpson(integrandwv1.real,x=fd)
integralwv2 = 4*sp.integrate.simpson(integrandwv2.real,x=fd)


print(max(intfft))
print(max(intfft)/(intfftwv1[0]*intfftwv2[0]))
print(integral)
print(integral/(integralwv1*integralwv2))

In [ ]:
phasearr = lambda n: np.exp(-2*np.pi*1j*n*dt*fd)
dom = np.arange(-20,20)
ran = [4*sp.integrate.simpson((integrand*phasearr(n)).real,x=fd) for n in dom]
plt.plot(dom, ran)
plt.show()
plt.plot(intfft[:20])

In [ ]:
freqs=np.fft.fftfreq(len(wv1))
freqs=freqs[:int(len(freqs)/2)]
wv1f = np.fft.fft(wv1)[:int(len(freqs))]
wv2f = np.fft.fft(wv2)[:int(len(freqs))]
lisasen = LisaSensitivity(freqs)

In [ ]:
def mollifier(x,f1,f2,n):
    if f1<x<f2:
        return np.exp(-1/(((f2-f1)/2)**n - (x - (f2+f1)/2)**n) + (2/(f2-f1))**n)
    else:
        return 0
def tophat(x,f1,f2):
    if f1<x<f2: return 1
    else: return 0


In [ ]:
dom = np.arange(1e-5,1,1e-5)
ran = LisaSensitivity(dom)
fig,ax = plt.subplots(4,1,figsize=(20,8))
fig.tight_layout()
plt.subplots_adjust(hspace=0.4)
ax[0].loglog(dom,ran)
ax[0].set_title("lisa sens")
ax[1].plot(freqs,wv1f)
ax[1].set_title("wv1f")
ax[2].plot(freqs, wv1f*np.conjugate(wv2f)/lisasen)
ax[2].set_title("h1. h2* / PSD");
ran = [tophat(x,0.0001,1.) for x in freqs]
ax[3].plot(freqs, wv1f*np.conjugate(wv2f)/lisasen *ran)
ax[3].set_title("h1. h2* / PSD * W");

In [ ]:
fig,ax = plt.subplots(4,1,figsize=(16,8))
ax[0].plot(freqs,np.fft.ifft(wv1f*np.conjugate(wv2f)/lisasen *ran))
ax[1].plot(freqs,np.fft.ifft(wv1f*np.conjugate(wv2f)/lisasen ))
ax[2].plot(freqs,np.fft.ifft(wv1f*np.conjugate(wv2f)/lisasen *ran)-np.fft.ifft(wv1f*np.conjugate(wv2f)/lisasen))
ax[3].plot(freqs[:10],np.fft.ifft(ran)[:10])

In [ ]:
inx=(0,400)
dom = td[inx[0]:inx[1]]
ran1 = (wv1[inx[0]:inx[1]]).real
for i in np.linspace(1,8,20):
    ran2 = (wv1[inx[0]:inx[1]]*np.exp(1j*np.pi*i/4)).real
    interp1 = sp.interpolate.CubicSpline(dom,ran1)
    interp2 = sp.interpolate.CubicSpline(dom,ran2)
    domain =np.arange(dom[0], dom[-1], (dom[-1]-dom[0])/len(dom)/100)
    peaks = sp.signal.find_peaks(ran1)[0]
    peaksdom = np.array(dom)[peaks]
    peaksdat = np.array(ran1)[peaks]
    interpp = sp.interpolate.CubicSpline(peaksdom, peaksdat)
    plt.plot(domain,interp1(domain))
    plt.plot(domain,interp2(domain))
    #plt.plot(domain, interpp(domain))

In [ ]:
fai = []
for i in np.linspace(1.e-16,4.5e-16,30):
    print(alphavalue(M,i))
    print("wv1")
    wv1 = withoutprocagen(M, m, a, p0, e0, Y0, qS, phiS, qK, phiK, dist,Phi_phi0=Phi_phi0, Phi_theta0=Phi_theta0, Phi_r0=Phi_r0, mich=mich, dt=dt, T=T)
    print('wv2')
    wv2 = withprocagen(M,m,i,a,p0,e0,Y0,T=T,qS=qS,phiS=phiS,qK=qK,phiK=phiK,dist=dist,mich=mich, UltralightBoson=ulb)
    
    if len(wv1)<len(wv2):
        wv1 = np.pad(wv1, (0,len(wv2)-len(wv1)))
    elif len(wv2)<len(wv1):
        wv2= np.pad(wv2, (0,len(wv1)-len(wv2)))
    
    minlen = min([len(wv1), len(wv2)])
    td = np.arange(minlen)*dt
    #wv1 = wv1[:minlen]
    #wv2 = wv2[:minlen]
    print("Faithfulness")
    fai.append([alphavalue(M,i),
        Faithfulness(td,wv1,wv2)]
    )
    print("Proca mass: "+str(i)+"\n\tfaithfulness: "+str(fai[-1]))
fai = np.asarray(fai)

In [ ]:
plt.plot(np.array(fai)[:,0],np.array(fai)[:,1])

In [ ]:
import sys

# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)